### FUZZYFIKASI

Fungsi Keanggotaan Umum

In [1]:
# fungsi keanggotaan segitiga
def keanggotaan_segitiga(x, a, b, c):
    if x <= a or x >= c:
        return 0
    elif x == b:
        return 1
    elif a < x < b:
        return (x - a) / (b - a)
    elif b < x < c:
        return (c - x) / (c - b)

# fungsi keanggotaan trapesium
def keanggotaan_trapesium(x, a, b, c, d):
    if x <= a or x >= d:
        return 0
    elif b <= x <= c:
        return 1
    elif a < x < b:
        return (x - a) / (b - a)
    elif c < x < d:
        return (d - x) / (d - c)

Fungsi Keanggotaan tiap Variabel

In [2]:
# fuzzifikasi variabel waktu_belajar
def fuzzifikasi_waktu_belajar(x):
    kecil = keanggotaan_trapesium(x, 0, 0, 356, 454)
    sedang = keanggotaan_segitiga(x, 356, 454, 527)
    besar = keanggotaan_trapesium(x, 454, 527, 3600, 3600)
    return {'kecil': kecil, 'sedang': sedang, 'besar': besar}

# fuzzifikasi variabel lama_jawab_soal
def fuzzifikasi_lama_jawab_soal(x):
    kecil = keanggotaan_trapesium(x, 0, 0, 367, 621)
    sedang = keanggotaan_segitiga(x, 367, 621, 1056)
    besar = keanggotaan_trapesium(x, 621, 1056, 9000, 9000)
    return {'kecil': kecil, 'sedang': sedang, 'besar': besar}

# fuzzifikasi variabel nilai
def fuzzifikasi_nilai(x):
    kecil = keanggotaan_trapesium(x, 0, 0, 66.7, 83)
    sedang = keanggotaan_segitiga(x, 66.7, 83, 100)
    besar = keanggotaan_segitiga(x, 83, 100, 100)
    return {'kecil': kecil, 'sedang': sedang, 'besar': besar}

In [3]:
x = fuzzifikasi_waktu_belajar(590)
y = fuzzifikasi_lama_jawab_soal(452)
z = fuzzifikasi_nilai(75)

print(f"waktu belajar   : {x}")
print(f"lama jawab soal : {y}")
print(f"nilai           : {z}")

waktu belajar   : {'kecil': 0, 'sedang': 0, 'besar': 1}
lama jawab soal : {'kecil': 0.6653543307086615, 'sedang': 0.3346456692913386, 'besar': 0}
nilai           : {'kecil': 0.49079754601227005, 'sedang': 0.50920245398773, 'besar': 0}


### SISTEM PAKAR

In [4]:
import mysql.connector

In [5]:
# koneksi ke database
def connect_to_db():
    return mysql.connector.connect(
        host="localhost",
        user="root",   
        password=" ",  
        database="sistem_pakar"
    )

# membaca rules dari database
def fetch_rules():
    conn = connect_to_db()
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM rules")
    rules = cursor.fetchall()
    conn.close()
    return rules

# menghitung certainty factor
def calculate_cf(rule, fuzzifikasi):
    cf = rule['kepercayaan_pakar'] / 100
    cf_value = (
        cf * fuzzifikasi[rule['rerata_durasi_akses_materi']] *
        fuzzifikasi[rule['durasi_berlatih']] *
        fuzzifikasi[rule['nilai']]
    )
    return cf_value

# forward chaining
def forward_chaining(fuzzifikasi):
    rules = fetch_rules()
    max_cf_lulus = 0
    max_cf_tidak_lulus = 0

    for rule in rules:
        cf_value = calculate_cf(rule, fuzzifikasi)
        if rule['kelulusan'] == "lulus":
            max_cf_lulus = max(max_cf_lulus, cf_value)
        else:
            max_cf_tidak_lulus = max(max_cf_tidak_lulus, cf_value)

    return "Lulus" if max_cf_lulus >= max_cf_tidak_lulus else "Tidak Lulus"

In [6]:
def main():
    # Koneksi ke database dan ambil data uji
    conn = connect_to_db()
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM dataset")
    dataset = cursor.fetchall()

    for data in dataset:
        rerata_durasi = data[1]
        durasi_berlatih = data[2]
        nilai = data[3]

        # Lakukan fuzzifikasi
        fuzzifikasi = {
            'rerata_durasi_akses_materi': fuzzifikasi_waktu_belajar(rerata_durasi),
            'durasi_berlatih'           : fuzzifikasi_lama_jawab_soal(durasi_berlatih),
            'nilai'                     : fuzzifikasi_nilai(nilai)
        }

        # Tentukan kelulusan
        hasil = forward_chaining(fuzzifikasi)
        print(f"ID: {data[0]} - Kelulusan: {hasil}")

    conn.close()

main()


ProgrammingError: 1045 (28000): Access denied for user 'root'@'localhost' (using password: YES)